In [198]:
import pandas as pd
import numpy as np
import gzip
import json
import re
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import math
sns.set()


%matplotlib inline

Collecting the most populated three tags for the science fiction books and their corresponding populations.

In [2]:
books_sci = pd.read_csv('books-scifi.csv')
books_scifiallcol = pd.read_csv('books-scifi_year_allcols.csv')

In [48]:
def clean_data_nasty_tags(dfin):

    popular_shelves = dfin['popular_shelves']
    count_shelves = dfin['count_shelves']
    
    popnew = []
    cnew = []

    for elm_str, elmc_str in zip(popular_shelves, count_shelves):    
        to_remove = []
        elmlist=eval('['+elm_str+']')
        elmclist=eval('['+elmc_str+']')
        elm = [i for all in elmlist for i in all]
        elmc = [i for all in elmclist for i in all]
        for i in range(len(elm)):
            if (len(elm[i]) == 1) or (bool(re.match('.*book*', elm[i])) == True)\
            or (bool(re.match('.*read*', elm[i])) == True) or (bool(re.match('.*favorite*', elm[i])) == True)\
            or (bool(re.match('.*need*', elm[i])) == True) or (bool(re.match('.*own*', elm[i])) == True)\
            or (bool(re.match('.*shelve*', elm[i])) == True) or (bool(re.match('.*like*', elm[i])) == True)\
            or (bool(re.match('.*shelf*', elm[i])) == True) or (bool(re.match('.*buy*', elm[i])) == True)\
            or (bool(re.match('tbr', elm[i])) == True) or (bool(re.match('.*finish*', elm[i])) == True)\
            or (bool(re.match('.*kindle*', elm[i])) == True) or (bool(re.match('.*list*', elm[i])) == True)\
            or (bool(re.match('.*year*', elm[i])) == True) or (bool(re.match('.*audio*', elm[i])) == True)\
            or (bool(re.match('.*library*', elm[i])) == True):
                #print(elm[i])
                to_remove.append(i) 
        
        popnew.append([elm[e] for e in range(len(elm)) if e not in to_remove])
        cnew.append([elmc[e] for e in range(len(elmc)) if e not in to_remove])
 
    return popnew, cnew

In [54]:
popnew, cnew = clean_data_nasty_tags(books_scifiallcol)


In [172]:
new_column_p = pd.Series(popnew, name='popular_shelves', index=range(len(popnew)))
new_column_c = pd.Series(cnew, name='count_shelves', index=range(len(cnew)))

df_copy = books_scifiallcol.copy()
df_copy.update(new_column_p)
df_copy.update(new_column_c)

In [173]:
books_scifiallcol.head(3)

,average_rating,book_id,count_shelves,isbn,num_pages,popular_shelves,publication_year,ratings_count,title
0,3.48,7203847,"['7615', '1526', '1108', '866', '321', '317', ...",0142427896,1.0,"['to-read', 'horror', 'stephen-king', 'fiction...",2010.0,45,The Tommyknockers
1,4.04,12182387,"['1057', '101', '59', '58', '55', '36', '35', ...",NaN,NaN,"['to-read', 'young-adult', 'ya', 'paranormal',...",NaN,4,"The Passion (Dark Visions, #3)"
2,3.80,20135365,"['1010', '17', '14', '13', '13', '12', '12', '...",NaN,NaN,"['to-read', 'sci-fi', 'young-adult', 'dystopia...",NaN,5,Hope's Daughter


In [174]:
df_copy.head(3)

,average_rating,book_id,count_shelves,isbn,num_pages,popular_shelves,publication_year,ratings_count,title
0,3.48,7203847,"[1526, 1108, 866, 317, 240, 231, 170, 160, 108...",0142427896,1.0,"[horror, stephen-king, fiction, science-fictio...",2010.0,45,The Tommyknockers
1,4.04,12182387,"[101, 59, 58, 55, 35, 26, 25, 19, 16, 16, 14, ...",NaN,NaN,"[young-adult, young-adult, paranormal, fantasy...",NaN,4,"The Passion (Dark Visions, #3)"
2,3.80,20135365,"[17, 14, 13, 13, 12, 12, 6, 5, 3, 3, 2, 2, 2, ...",NaN,NaN,"[science-fiction, young-adult, dystopia, dysto...",NaN,5,Hope's Daughter


In [175]:
def replace_tags(dfin):
    popular_shelves = dfin['popular_shelves']
    count_shelves = dfin['count_shelves']    
    
    popnew = []
    cnew = []

    for elm, elmc in zip(popular_shelves, count_shelves):    
        for i in range(len(elm)):
            elm[i] = elm[i].replace('classics', 'classic')
            elm[i] = elm[i].replace('scifi', 'science-fiction')
            elm[i] = elm[i].replace('sci-fi', 'science-fiction')
            elm[i] = elm[i].replace('ya', 'young-adult')

        popnew.append([elm[e] for e in range(len(elm))])
        cnew.append([elmc[e] for e in range(len(elmc))])
    
    return popnew, cnew

In [308]:
p, c = replace_tags(df_copy)
new_column_p = pd.Series(p, name='popular_shelves', index=range(len(p)))
new_column_c = pd.Series(c, name='count_shelves', index=range(len(c)))
df_copy.update(new_column_p)
df_copy.update(new_column_c)

In [322]:
def remove_sf_tags(dfin):

    popular_shelves = dfin['popular_shelves']
    count_shelves = dfin['count_shelves']
    
    popnew = []
    cnew = []

    for elm, elmc in zip(popular_shelves, count_shelves):    
        to_remove = []
        for i in range(len(elm)):
            if (bool(re.match('.*science-fiction*', elm[i])) == True):
                to_remove.append(i) 
        popnew.append([elm[e] for e in range(len(elm)) if e not in to_remove])
        cnew.append([elmc[e] for e in range(len(elmc)) if e not in to_remove])
 
    return popnew, cnew

In [323]:
p2, c2 = remove_sf_tags(df_copy)


In [325]:
new_column_p = pd.Series(p2, name='popular_shelves', index=range(len(p)))
new_column_c = pd.Series(c2, name='count_shelves', index=range(len(c)))
df_copy.update(new_column_p)
df_copy.update(new_column_c)
df_copy.head(3)

,average_rating,book_id,count_shelves,isbn,num_pages,popular_shelves,publication_year,ratings_count,title
0,3.48,7203847,"[1526, 1108, 866, 231, 170, 160, 108, 99, 74, ...",0142427896,1.0,"[horror, stephen-king, fiction, default, thril...",2010.0,45,The Tommyknockers
1,4.04,12182387,"[101, 59, 58, 55, 35, 26, 25, 19, 16, 16, 14, ...",NaN,NaN,"[young-adult, young-adult, paranormal, fantasy...",NaN,4,"The Passion (Dark Visions, #3)"
2,3.80,20135365,"[14, 13, 13, 12, 6, 5, 3, 3, 2, 2, 2, 2, 2, 2,...",NaN,NaN,"[young-adult, dystopia, dystopian, romance, fa...",NaN,5,Hope's Daughter


In [326]:
def merge_similar_tags(dfin):
    popular_shelves = dfin['popular_shelves']
    count_shelves = dfin['count_shelves']    
    
    popnew = []
    cnew = []
    sum_tags = []

    for elm, elmc in zip(popular_shelves, count_shelves):  

        to_remove = []
        elmc_int = np.array([int(i) for i in elmc])
        rep = [item for item, count in Counter(elm).items() if count > 1]
        if rep:
            to_remove = [elm.index(tag) for tag in rep]
            for r in rep:
                rep_idx = [i for i,d in enumerate(elm) if d==r]
                sum_rep = (np.sum(elmc_int[rep_idx]))
                elmc_int[rep_idx[1]] = sum_rep
            
        popnew.append([elm[e] for e in range(len(elm)) if e not in to_remove])
        cnew.append([elmc_int[e] for e in range(len(elmc_int)) if e not in to_remove])
        sum_tags.append(sum([elmc_int[e] for e in range(len(elmc_int)) if e not in to_remove]))
    return popnew, cnew, sum_tags    

In [338]:
p, c, sum_tags = merge_similar_tags(df_copy)

In [339]:
df_copy2 = df_copy.copy()
new_column_p = pd.Series(p, name='popular_shelves', index=range(len(p)))
new_column_c = pd.Series(c, name='count_shelves', index=range(len(c)))
df_copy2.update(new_column_p)
df_copy2.update(new_column_c)

In [340]:
df_copy2.head(3)

,average_rating,book_id,count_shelves,isbn,num_pages,popular_shelves,publication_year,ratings_count,title
0,3.48,7203847,"[1526, 1108, 866, 231, 170, 160, 108, 99, 74, ...",0142427896,1.0,"[horror, stephen-king, fiction, default, thril...",2010.0,45,The Tommyknockers
1,4.04,12182387,"[160, 58, 55, 35, 26, 25, 19, 16, 16, 14, 14, ...",NaN,NaN,"[young-adult, paranormal, fantasy, romance, ur...",NaN,4,"The Passion (Dark Visions, #3)"
2,3.80,20135365,"[13, 13, 12, 6, 19, 3, 3, 2, 2, 2, 2, 2, 2, 2,...",NaN,NaN,"[dystopia, dystopian, romance, fantasy, young-...",NaN,5,Hope's Daughter


In [341]:
df_copy2['count tags'] = sum_tags

In [342]:
df_copy2.head(3)

,average_rating,book_id,count_shelves,isbn,num_pages,popular_shelves,publication_year,ratings_count,title,count tags
0,3.48,7203847,"[1526, 1108, 866, 231, 170, 160, 108, 99, 74, ...",0142427896,1.0,"[horror, stephen-king, fiction, default, thril...",2010.0,45,The Tommyknockers,5483
1,4.04,12182387,"[160, 58, 55, 35, 26, 25, 19, 16, 16, 14, 14, ...",NaN,NaN,"[young-adult, paranormal, fantasy, romance, ur...",NaN,4,"The Passion (Dark Visions, #3)",594
2,3.80,20135365,"[13, 13, 12, 6, 19, 3, 3, 2, 2, 2, 2, 2, 2, 2,...",NaN,NaN,"[dystopia, dystopian, romance, fantasy, young-...",NaN,5,Hope's Daughter,126


In [349]:
# First take only first x tags
def get_first_tags(dfin, cutoff):
    popular_shelves = dfin['popular_shelves']
    count_shelves = dfin['count_shelves']    
    popnew = []
    cnew = []

    for elm, elmc in zip(popular_shelves, count_shelves):    
        listp = [elm[e] for e in range(len(elm))][0:int(cutoff)]
        listc = [elmc[e] for e in range(len(elmc))][0:int(cutoff)]
        if len(listp) == 0:
            popnew.append('none')
            cnew.append(0)
        else:
            popnew.append(listp[0])
            cnew.append(listc[0])
    
    return popnew, cnew

In [350]:
p3, c3 = get_first_tags(df_copy2, 1)


In [351]:
new_column_p = pd.Series(p3, name='popular_shelves', index=range(len(p)))
new_column_c = pd.Series(c3, name='count_shelves', index=range(len(c)))
df_copy2.update(new_column_p)
df_copy2.update(new_column_c)

In [352]:
df_copy2.head(3)

,average_rating,book_id,count_shelves,isbn,num_pages,popular_shelves,publication_year,ratings_count,title,count tags
0,3.48,7203847,1526,0142427896,1.0,horror,2010.0,45,The Tommyknockers,5483
1,4.04,12182387,160,NaN,NaN,young-adult,NaN,4,"The Passion (Dark Visions, #3)",594
2,3.80,20135365,13,NaN,NaN,dystopia,NaN,5,Hope's Daughter,126


In [353]:
df_book_year_tag = df_copy2.drop(df_copy2.columns[[2, 3, 4, 7, 8, 9]], axis=1)

In [354]:
df_book_year_tag.head(4)

,average_rating,book_id,popular_shelves,publication_year
0,3.48,7203847,horror,2010.0
1,4.04,12182387,young-adult,NaN
2,3.80,20135365,dystopia,NaN
3,3.43,780918,historical-fiction,NaN


In [355]:
df_book_year_tag.to_csv('book_year_1tag.csv', index=False)

In [294]:
print(type(df_copy2['popular_shelves'][0]))

<class 'str'>


In [295]:
df_copy2[df_copy2['publication_year'] == 14]

,average_rating,book_id,count_shelves,isbn,num_pages,popular_shelves,publication_year,ratings_count,title,count tags
21928,4.0,23126275,2,NaN,302.0,science-fiction,14.0,24,On The Run (The Moriya Chronicles: Book 1),18


Now that we have the three most relevant tags for each book, let's convert the tag name into a numerical value.

In [257]:
category_frequency = []
num_tags = []

for i in range(df_copy2.shape[0]):
    each_line = df_copy2['popular_shelves'][i]
    for content in each_line:
        category_frequency.append(content)
        num_tags.append(len(content))

In [258]:
print(category_frequency[0],num_tags[0])

horror 6


In [262]:
new_feature = Counter(category_frequency).most_common(100)

feature = pd.DataFrame()
for ind,val in enumerate(new_feature[2:]):
    feature.loc[ind,'Feature'] = val[0]
    feature.loc[ind,'IDF'] =  math.log1p(len(df_copy2.book_id) / val[1])

In [263]:
feature.head(4)

,Feature,IDF
0,fantasy,1.410292
1,horror,2.476973
2,young-adult,2.591937
3,comics,2.616929


In [270]:
feature[feature['IDF'] == 5.499532]

,Feature,IDF


In [271]:
feature

,Feature,IDF
0,fantasy,1.410292
1,horror,2.476973
2,young-adult,2.591937
3,comics,2.616929
4,default,2.620232
5,graphic-novels,2.730553
6,romance,2.739848
7,paranormal,2.846486
8,short-stories,3.053630
9,mystery,3.108750


In [265]:
len(category_frequency)

326536

In [203]:
len(df_copy2)

92803

In [266]:
Counter(feature.IDF)

Counter({1.4102915888714314: 1,
         2.4769727921236036: 1,
         2.5919366537796895: 1,
         2.6169294733703388: 1,
         2.6202323593679306: 1,
         2.730553047861989: 1,
         2.7398476730049266: 1,
         2.8464859085597163: 1,
         3.053629583668823: 1,
         3.1087501188803532: 1,
         3.256057653629862: 1,
         3.2609814048590637: 1,
         3.296470043486442: 1,
         3.346665722308471: 1,
         3.390117796408945: 1,
         3.483917551113216: 1,
         3.5100099739990993: 1,
         3.5594498028722654: 1,
         3.562313728985269: 1,
         3.7006558621865864: 1,
         3.742636961166146: 1,
         3.959725047451519: 1,
         3.964084935297429: 1,
         3.9701122028858102: 1,
         4.0544993154639615: 1,
         4.113240302198315: 1,
         4.199874914836857: 2,
         4.216751188513698: 1,
         4.217460773532061: 1,
         4.275975683642587: 2,
         4.383474860850247: 1,
         4.42036412944988